# Registration of Scanned Maps and Species Hotspots onto World Map for Half Earth Project Using Computer Vision
The sample code uses Computer Vision techniques to automatically register scanned images of any part of the world onto the world map and hence overlay the species hotspots depicted in the scanned images onto the world map.

## Pre-requisites
OpenCV library: This sample requires installation of OpenCV library. Kindly install OpenCV using the following command:
pip install opencv-contrib-python

## Data Used
The data used for this sample are the scanned images from Half Earth Project and the corresponding species region masks.

## Defining the constants

In [1]:
# Path to the folder where the scanned map images are placed
IMAGES_TO_REGISTER_FOLDER = r"C:\Users\ati11038\Documents\workspace\data\half_earth\extracted_samples\extracted_maps"

# Path to the folder where the species region mask images are placed
IMAGES_TO_REGISTER_SPECIES_MASKS_FOLDER = r"C:\Users\ati11038\Documents\workspace\data\half_earth\extracted_samples" \
                                          r"\extracted_masks"

# File path and filename of world image file 
WORLD_MAP_IMAGE_PATH = r"C:\Users\ati11038\Documents\workspace\data\half_earth\world_map"
WORLD_MAP_IMAGE_NAME = "map_without_rivers.tif"

# Path to the folder where the outputs need to appear
RESULTS_FOLDER = r"C:\\Users\\ati11038\\Documents\\workspace\\data\\half_earth\\results\\main\\22_4_2020" \
                 r"\\3"

# Minimum expected IOU of homography transform result with default homography to qualify as improved homography
# (float: 0.0 - 1.0)
IOU_THRESHOLD_HOMOGRAPHY = 0.85

# Minimum expected spread of estimated control points over the scanned image
# (float: 0.0 - 1.0)
CONTOUR_RECT_AREA_RATIO_THRESHOLD = 0.40

# Threshold for maximum allowed distance of control points from the nearest contour point for refinement
# (int: 1 - 10) 
REFINEMENT_DISTANCE_THRESHOLD = 5

# Minimum threshold for segmenting mask from species region mask image
# (int: 1 - 255)
SPECIES_REGION_MASK_THRESHOLD = 50
SPECIES_REGION_MASK_VALUE = 255

# Upper and lower thresholds for homogenizing query image
HOMOGENIZE_UPPER_THRESHOLD = 245
HOMOGENIZE_LOWER_THRESHOLD = 180

# Boundary values for Latitude and Longitude corresponding to world map image
LONGITUDE_EXTREME_LEFT = -180
LATITUDE_EXTREME_TOP = 85
LONGITUDE_EXTREME_RIGHT = 180
LATITUDE_EXTREME_BOTTOM = -85

# Maximum number of control points needed 
MAX_CONTROL_POINTS = 8